In [ ]:
%config Completer.use_jedi = False
import cv2
import mediapipe
import pandas as pd
import numpy as np
import csv
from os import listdir
from os.path import isfile, join
  
#initialize modules
drawingModule = mediapipe.solutions.drawing_utils
handsModule = mediapipe.solutions.hands

In [ ]:
#list all videos in mediafolder
mypath = "./MediaToAnalyze/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))] # get all files that are in mediatoanalyze
#time series output folder
foldtime = "./Timeseries_Output/"

In [ ]:
#some preperatory functions and lists for saving the data


#take some google classification object and convert it into a string
def makegoginto_str(gogobj):
    gogobj = str(gogobj).strip("[]")
    gogobj = gogobj.split("\n")
    return(gogobj[:-1]) #ignore last element as this has nothing

#Hand landmarks
markers = ['WRIST', 'THUMB_CMC', 'THUMB_MCP', 'THUMB_IP', 'THUMB_TIP', 
 'INDEX_MCP', 'INDEX_PIP', 'INDEX_DIP', 'INDEX_TIP', 
 'MIDDLE_MCP', 'MIDDLE_PIP', 'MIDDLE_DIP','MIDDLE_TIP', 
 'RING_MCP', 'RING_TIP', 'RING_DIP', 'RING_TIP', 
 'PINKY_MCP', 'PINKY_PIP', 'PINKY_DIP', 'PINKY_TIP']

#make the stringifyd position traces into clean values
def listpostions(newsamplemaerks):
    tracking_p = []
    for value in newsamplelmarks:
        stripped = value.split(':', 1)[1]
        stripped = stripped.strip() #remove spaces in the string if present
        tracking_p.append(stripped) #add to this list  
    return(tracking_p)

#a function that only retrieves the numerical info in a string
def only_numerics(seq):
    seq_type= type(seq)
    return seq_type().join(filter(seq_type.isdigit, seq))   

In [ ]:
#loop through the frames of the video
for ff in onlyfiles:
    capture = cv2.VideoCapture(mypath+ff)
    frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = capture.get(cv2.CAP_PROP_FPS)

    print(frameWidth, frameHeight, fps )

    #hands tracking with keypoints save!
    #make a video file
    samplerate = fps #make the same as current 
    fourcc = cv2.VideoWriter_fourcc(*'XVID') #(*'XVID')
    out = cv2.VideoWriter('Videotracking_output/'+ff[:-4]+'.avi', fourcc, fps= samplerate, frameSize = (int(frameWidth), int(frameHeight))) #make sure that frameheight/width is the same a original

    #make a variable list with x, y, z, info where data is appended to
    markerxyz = []
    for mark in markers:
        for pos in ['X', 'Y', 'Z']:
            nm = pos + "_" + mark
            markerxyz.append(nm)
    addvariable = ['index', 'confidence', 'hand', 'time']
    addvariable.extend(markerxyz)
    time = 0
    fr = 1
    timeseries = [addvariable]
    #MAIN ROUTINE
    with handsModule.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.75, max_num_hands=6) as hands:
         while (True):
            ret, frame = capture.read()
            if ret == True:
                results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                if results.multi_hand_landmarks != None:
                    #attach an id based on location                    
                    for handLandmarks, handinfo in zip(results.multi_hand_landmarks,results.multi_handedness):
                        newsamplelmarks = makegoginto_str(handLandmarks.landmark)
                        newsamplelmarks = listpostions(newsamplelmarks)
                        newsampleinfo = makegoginto_str(handinfo) #get info the hands
                        fuldataslice = [fr, newsampleinfo[2], newsampleinfo[3]]
                        fuldataslice.extend([str(time)]) #add time
                        fuldataslice.extend(newsamplelmarks) #add positions
                        timeseries.append(fuldataslice)
                        #get information about hand index [0], hand confidence [1], handedness [2]              
                        for point in handsModule.HandLandmark:
                            normalizedLandmark = handLandmarks.landmark[point]
                            pixelCoordinatesLandmark = drawingModule._normalized_to_pixel_coordinates(normalizedLandmark.x, normalizedLandmark.y, frameWidth, frameHeight)
                            cv2.circle(frame, pixelCoordinatesLandmark, 5, (0, 255, 0), -1)
                if results.multi_hand_landmarks == None:
                    timeseries.append(["NA"]) #add a row of NAs
                cv2.imshow('Test hand', frame)
                out.write(frame)  #########################################comment this out if you dont wn
                time = round(time+1000/samplerate)
                fr = fr+1
                if cv2.waitKey(1) == 27:
                    break
            if ret == False:
                break
    out.release()
    capture.release()
    cv2.destroyAllWindows()

    ####################################################### data to be written row-wise in csv fil
    data = timeseries

    # opening the csv file in 'w+' mode
    file = open(foldtime+ff[:-4]+'.csv', 'w+', newline ='')
    #write it
    with file:    
        write = csv.writer(file)
        write.writerows(data)

In [ ]:
##a tricky part of the the output when working with multiple persons is that you get a variable number of hand samples
#vidpath = "./Videotracking_output/"
#vfiles = [f for f in listdir(vidpath) if isfile(join(vidpath, f))] # get all files that are in mediatoanalyze
#
##set the font for the text on the image
#font                   = cv2.FONT_HERSHEY_SIMPLEX
#fontScale              = 0.5
#fontColor              = (255,255,255)
#lineType               = 2
#
##loop through the frames of the video
#for ff in vfiles:
#    capture = cv2.VideoCapture(vidpath+ff)
#    frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
#    frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
#    fps = capture.get(cv2.CAP_PROP_FPS)
#    #make a new video with identification labels
#    samplerate = fps #make the same as current 
#    fourcc = cv2.VideoWriter_fourcc(*'XVID') #(*'XVID')
#    out = cv2.VideoWriter('Videotracking_output_withIDs/'+ff[:-4]+'_IDed.avi', fourcc, fps= samplerate, frameSize = (int(frameWidth), int(frameHeight))) #make sure that frameheight/width is the same a original
#    #load in relevant timeseries
#    dftime = pd.read_csv(foldtime+ff[:-4]+'.csv')
#    ffr = 1 #start with time
#    while (True):
#        ret, frame = capture.read()
#        #get selection time series
#        if ret == True:
#            slicecur = dftime.loc[dftime['index'] == ffr]
#            ffr = ffr + 1
#            #loop through rows of time series
#            for row in slicecur.iterrows():
#                #take one point of the hand and draw a number on it
#                pixelCoordinatesLandmark = drawingModule._normalized_to_pixel_coordinates(row[1]['X_WRIST'], row[1]['Y_WRIST'], frameWidth, frameHeight)
#                cv2.putText(frame, "ID", pixelCoordinatesLandmark, font, fontScale, fontColor, lineType)      
#        #show the image
#            #cv2.imshow('test', frame)
#            out.write(frame) 
#        #save the previous slice
#        sliceprev = slicecur   
#        #if there are no frames go out of the loop
#        if ret == False:
#            break   
#
#out.release()
#capture.release()
#cv2.destroyAllWindows()
# 

In [ ]:
##check clostest distance to the corner
#slicecur = dftime.loc[dftime['index'] == 426] #select the current frame
#slicecur.sort_values(["X_WRIST", "Y_WRIST"], ascending = (True, True)) #order the dataframe based on wrist x and y
#
#right['person'] = list(range(1, rghth+1))
#left['person'] = list(range(1, lefth+1))
## Using 'Address' as the column name
## and equating it to the list
#
#
#